<a href="https://colab.research.google.com/github/defyMiy/Intern/blob/main/PartOfSpeech(CRF%2COrchid).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Jul  4 14:31:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# dataset
!wget https://raw.githubusercontent.com/tchayintr/thtb/master/thtb_orchidpp.txt

--2022-07-04 14:31:24--  https://raw.githubusercontent.com/tchayintr/thtb/master/thtb_orchidpp.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2064161 (2.0M) [text/plain]
Saving to: ‘thtb_orchidpp.txt’

thtb_orchidpp.txt   100%[===================>]   1.97M  --.-KB/s    in 0.04s   

2022-07-04 14:31:24 (46.7 MB/s) - ‘thtb_orchidpp.txt’ saved [2064161/2064161]



In [3]:
!pip install sklearn_crfsuite
!pip install scikit-learn==0.22.2
!pip install nltk pythainlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 965 kB 7.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.1 MB 9.0 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.22.2 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.22.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████████████████

In [4]:
# https://pythainlp.github.io/dev-docs/api/tag.html
POS_TAG = {
    'NOUN' : ['NOUN', 'NCMN', 'NTTL', 'CNIT', 'CLTV', 'CMTR', 'CFQC', 'CVBL'],
    'VERB' : ['VACT', 'VSTA'],
    'PROPN' : ['PROPN', 'NPRP'],
    'ADJ' : ['ADJ', 'NONM', 'VATT', 'DONM'],
    'ADV' : ['ADV', 'ADVN', 'ADVI', 'ADVP', 'ADVS'],
    'INTJ' : ['INT'],
    'PRON' : ['PRON', 'PPRS', 'PDMN', 'PNTR'],
    'DET' : ['DET', 'DDAN', 'DDAC', 'DDBQ', 'DDAQ', 'DIAC', 'DIBQ', 'DIAQ'],
    'NUM' : ['NUM', 'NCNM', 'NLBL', 'DCNM'],
    'AUX' : ['AUX', 'XVBM', 'XVAM', 'XVMM', 'XVBB', 'XVAE'],
    'ADP' : ['ADP', 'RPRE'],
    'CCONJ' : ['CCONJ', 'JCRG'],
    'SCONJ' : ['SCONJ', 'PREL', 'JSBR', 'JCMP'],
    'PART' : ['PART', 'FIXN', 'FIXV', 'EAFF', 'EITT', 'NEG'],
    'PUNCT' : ['PUNCT', 'PUNC']
}

In [5]:
with open('thtb_orchidpp.txt', 'r') as f:
  sentences = f.read().split('\n')
  f.close()
len(sentences)

5001

In [6]:
import re
from IPython.display import clear_output

tagged_sentence = []

for num, sentence in enumerate(sentences):
  print('{:.2f}'.format(num * 100 / len(sentences)))
  tag_words = []
  words = sentence.split(' ')
  for i, word in enumerate(words):
    for key in POS_TAG:
      for value in POS_TAG.get(key):
        if '[' + value in word:
          if ']' not in words[i+1]:
            tag_word = (words[i+1] + ' ' + words[i+2].replace(']', ''), key)
          else:
            tag_word = (words[i+1].replace(']', ''), key)
          tag_words.append(tag_word)
  tagged_sentence.append(tag_words)
  clear_output()
print("Number of Tagged Sentences ", len(tagged_sentence))

tagged_words = [tup for sent in tagged_sentence for tup in sent]
print("Total Number of Tagged words", len(tagged_words))

vocab = set([word for word,tag in tagged_words])
print("Vocabulary of the Corpus", len(vocab))

tags = set([tag for word,tag in tagged_words])
print("Number of Tags in the Corpus ", len(tags))

Number of Tagged Sentences  5001
Total Number of Tagged words 86046
Vocabulary of the Corpus 6152
Number of Tags in the Corpus  14


In [7]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(tagged_sentence,test_size=0.2,random_state=1234)
print("Number of Sentences in Training Data ",len(train_set))
print("Number of Sentences in Testing Data ",len(test_set))

Number of Sentences in Training Data  4000
Number of Sentences in Testing Data  1001


In [8]:
def features(sentence,index):
    ### sentence is of the form [w1,w2,w3,..], index is the position of the word in the sentence
    return {
        'is_first_capital':int(sentence[index][0].isupper()),
        'is_first_word': int(index==0),
        'is_last_word':int(index==len(sentence)-1),
        'is_complete_capital': int(sentence[index].upper()==sentence[index]),
        'prev_word':'' if index==0 else sentence[index-1],
        'next_word':'' if index==len(sentence)-1 else sentence[index+1],
        'is_numeric':int(sentence[index].isdigit()),
        'is_alphanumeric': int(bool((re.match('^(?=.*[0-9]$)(?=.*[a-zA-Z])',sentence[index])))),
        'prefix_1':sentence[index][0],
        'prefix_2': sentence[index][:2],
        'prefix_3':sentence[index][:3],
        'prefix_4':sentence[index][:4],
        'suffix_1':sentence[index][-1],
        'suffix_2':sentence[index][-2:],
        'suffix_3':sentence[index][-3:],
        'suffix_4':sentence[index][-4:],
        'word_has_hyphen': 1 if '-' in sentence[index] else 0  
         }
def untag(sentence):
    return [word for word,tag in sentence]


def prepareData(tagged_sentences):
    X,y=[],[]
    for sentences in tagged_sentences:
        X.append([features(untag(sentences), index) for index in range(len(sentences))])
        y.append([tag for word,tag in sentences])
    return X,y
X_train,y_train=prepareData(train_set)
X_test,y_test=prepareData(test_set)

In [9]:
from sklearn_crfsuite import CRF

crf = CRF(
    algorithm='lbfgs',
    c1=0.01,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.01, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [10]:
from sklearn_crfsuite import metrics
from sklearn_crfsuite import scorers

y_pred=crf.predict(X_test)
print("F1 score on Test Data ")
print(metrics.flat_f1_score(y_test, y_pred,average='weighted',labels=crf.classes_))
print("F score on Training Data ")
y_pred_train=crf.predict(X_train)
metrics.flat_f1_score(y_train, y_pred_train,average='weighted',labels=crf.classes_)

### Look at class wise score
print(metrics.flat_classification_report(
    y_test, y_pred, labels=crf.classes_, digits=3
))

F1 score on Test Data 
0.9208074642392937
F score on Training Data 
              precision    recall  f1-score   support

        PART      0.948     0.876     0.910       893
        VERB      0.922     0.916     0.919      2814
         DET      0.941     0.947     0.944       304
         ADP      0.893     0.900     0.896      1476
        NOUN      0.925     0.923     0.924      4977
         ADV      0.936     0.952     0.944       690
       PROPN      0.895     0.798     0.844       193
       CCONJ      0.867     0.860     0.863       242
       SCONJ      0.872     0.865     0.869       772
         AUX      0.896     0.927     0.911       765
         NUM      0.903     0.888     0.895       365
        PRON      0.908     0.908     0.908        65
         ADJ      0.964     0.960     0.962       845
       PUNCT      0.930     0.959     0.944      2624

    accuracy                          0.921     17025
   macro avg      0.914     0.906     0.910     17025
weighted avg